In [1]:
!nvidia-smi

Tue Jan  9 11:47:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.6 MB/s 

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.5 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.0 MB/s eta 0:00:00
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk,load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

from tqdm import tqdm
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pagasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [7]:
## Download and unzip data

!wget https://github.com/Ankitzanzmera/data_repository/raw/main/summarizer-data.zip
!unzip summarizer-data.zip
!rm -rf summarizer-data.zip

--2024-01-09 11:49:16--  https://github.com/Ankitzanzmera/data_repository/raw/main/summarizer-data.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ankitzanzmera/data_repository/main/summarizer-data.zip [following]
--2024-01-09 11:49:16--  https://raw.githubusercontent.com/Ankitzanzmera/data_repository/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.06s   

2024-01-09 11:49:16 (127 MB/s) - ‘summarizer-data.zip’ saved [7903594/

In [8]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]
print(f' Split Lengths : {split_lengths}')

print(f"Features : {dataset_samsum['train'].column_names}")

print("\n Dialogue:")
print(dataset_samsum['test'][1]['dialogue'])

print("\n Summary : ")
print(dataset_samsum['test'][1]['summary'])

 Split Lengths : [14732, 819, 818]
Features : ['id', 'dialogue', 'summary']

 Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

 Summary : 
Eric and Rob are going to watch a stand-up on youtube.


In [10]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'],max_length = 128, truncation = True)

  return {
      "input_ids": input_encodings['input_ids'],
      "attention_mask":input_encodings['attention_mask'],
      "labels": target_encodings['input_ids']
  }

In [11]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [12]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [13]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pagasus)

In [14]:
from transformers import TrainingArguments,Trainer

trainer_args = TrainingArguments(
    output_dir = "pegasus_samsum",
    num_train_epochs = 10,
    warmup_steps = 500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay = 0.1,
    logging_steps = 10,
    evaluation_strategy="steps",
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)

In [15]:
trainer = Trainer(model = model_pagasus,
                  args = trainer_args,
                  tokenizer = tokenizer,
                  data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt['test'],
                  eval_dataset = dataset_samsum_pt['validation']
                  )

In [16]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.229800,1.556282


TrainOutput(global_step=510, training_loss=1.8607681106118594, metrics={'train_runtime': 1514.8015, 'train_samples_per_second': 5.407, 'train_steps_per_second': 0.337, 'total_flos': 3131112806301696.0, 'train_loss': 1.8607681106118594, 'epoch': 9.96})

In [17]:
## Evaluation

def generate_batch_size_chunks(list_of_elements,batch_size):
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = "article", column_summary = "highlights"):
    article_batches = list(generate_batch_size_chunks(dataset[column_text],batch_size))
    target_batches = list(generate_batch_size_chunks(dataset[column_summary],batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches,target_batches),total = len(article_batches)):
      inputs = tokenizer(article_batch,max_length = 1024,truncation = True, padding = "max_length",return_tensors = "pt")

      summaries = model.generate(input_ids = inputs['input_ids'].to(device),attention_mask = inputs['attention_mask'].to(device),length_penalty = 0.8,num_beams = 8,max_length = 128)

      decoded_summaries = [tokenizer.decode(s,skip_special_tokens = True,clean_up_tokenization_spaces = True) for s in summaries]

      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

      metric.add_batch(predictions = decoded_summaries, references = target_batch)

    score = metric.compute()
    return score


In [18]:
rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-18-cc41bb0ac104>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [19]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size = 2, column_text = "dialogue",column_summary = "summary"
)

rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = [f'pagasus'])

100%|██████████| 5/5 [00:10<00:00,  2.00s/it]


,rouge1,rouge2,rougeL,rougeLsum
pagasus,0.023791,0.0,0.023811,0.023838


In [24]:
## Save Model
# model_pagasus.save_pretrained("pegasus-samsum-model")
model_pagasus.save_pretrained("/content/drive/MyDrive/data/")

In [21]:
## Save Tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [22]:
## Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")
# pagasus_samsum_model = AutoModelForSeq2SeqLM.from_pretrained("/content/mydrive/MyDrive/data/pegasus-samsum-model")

In [23]:
## test

gen_kwargs = {"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text = dataset_samsum["test"][0]['dialogue']
reference = dataset_samsum['test'][0]['summary']

pipe = pipeline("summarization",model = "pegasus-samsum-model",tokenizer = tokenizer)
# pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail",tokenizer=tokenizer)


print("Dialogue:")
print(sample_text)

print("\n Reference Summary")
print(reference)

print("\n Model summary")
print(pipe(sample_text,**gen_kwargs)[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

 Reference Summary
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

 Model summary
Amanda can't find Betty's number, so she asks Larry. He called Betty last time they were at the park together. Hannah wants Amanda to text Larry.
